In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=True)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 7314.80it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4092.93it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2007840.19it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 7575.38it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4344.11it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 577097.76it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 7162.12it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 4128.01it/s]
Processing length_right with len: 100%|██████████| 1115/1115 [00:00<00:00, 437233.45it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 7249.01it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4103.22it/s]
Processing length_right with len: 100%|██████████| 2300/2300 [00:00<00:00, 570991.37it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.contrib.models.MatchLSTM()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['embedding_input_dim'] = preprocessor.context['vocab_size'] + 1
model.params['embedding_output_dim'] = 100
model.params['embedding_trainable'] = True
model.params['fc_num_units'] = 100
model.params['lstm_num_units'] = 100
model.params['dropout_rate'] = 0.5
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to MatchLSTM.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1667400     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
lstm_left (LSTM)                (None, 10, 100)      80400       embedding

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

862183424/862182613 [==============================] - 864s 1us/step


In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))

In [10]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=2, num_neg=1, batch_size=20)
len(train_generator)

102

In [12]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=5, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 19s 185ms/step - loss: 0.5881
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.616749 - normalized_discounted_cumulative_gain@5(0):0.677532 - mean_average_precision(0):0.630450
Epoch 2/30
102/102 [==============================] - 19s 185ms/step - loss: 0.5123
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.598433 - normalized_discounted_cumulative_gain@5(0):0.660294 - mean_average_precision(0):0.612336
Epoch 3/30
102/102 [==============================] - 19s 186ms/step - loss: 0.4486
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.613545 - normalized_discounted_cumulative_gain@5(0):0.671999 - mean_average_precision(0):0.625587
Epoch 4/30
 19/102 [====>.........................] - ETA: 15s - loss: 0.3945

Process ForkPoolWorker-140:
Process ForkPoolWorker-139:
Process ForkPoolWorker-138:
Process ForkPoolWorker-136:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/eshion/.pyenv/versions/3.6.5/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/eshion/.pyenv/versions/3.6.5/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/eshion/.pyenv/versions/3.6.5/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/eshion/.pyenv/versions/3.6.5/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/Users/eshion/.pyenv/versions/3.6.5/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/Users/eshion/.pyenv/versions/3.6.5/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._

KeyboardInterrupt: 